In [ ]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime


In [ ]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
print(tickers)

In [ ]:
# Get stock prices and returns
startdate = datetime.datetime(2020,3,23)
enddate = datetime.datetime(2020,3,27)

temp_tickers = ['MMM', 'ABT', 'ABBV']

file = open("stocks.csv",'w',newline='')
for ticker in temp_tickers:
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass

file.close()


In [ ]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'ELRou1VKpS5p7EBEGamXqqGRT'
consumer_secret = 'QbsToHJOyA9ZsDrFCCw1aErF9LubwqlfLpLcJviruhKepQwX3D'
access_key= '3138259520-6jOClvoHXzF5DCf2XaylTxhrn1Rb7X50ykQyU3C'
access_secret = 'r9bokM5UvRbS8QnsYdQksSi2tO8gRj7T4sfq2K41jzmQM'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

file = open("searchTweets.csv","w",newline='')
alltweets = csv.writer(file)

for ticker in tickers:
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search,q=query,lang='en',tweet_mode='extended').items(100)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass

file.close()


In [ ]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)

sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)

for row in alltweets:
    blob = TextBlob(row[3])
    if blob.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive"])
    elif blob.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative"])     
    elif blob.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral"])    
        
file.close()
sentiment_file.close()